# --- Day 7: Some Assembly Required ---

https://adventofcode.com/2015/day/7

## Get Input Data

In [1]:
wire_instructions = {}
with open('../inputs/wire_instructions.txt') as f:
    for line in f:
        _input, _output = line.rstrip().split(' -> ')
        wire_instructions[_output] = _input.split()

# Check out the first 15 items in the wire_instructions dictionary
list(wire_instructions.items())[:15]

[('dr', ['NOT', 'dq']),
 ('kh', ['kg', 'OR', 'kf']),
 ('eq', ['ep', 'OR', 'eo']),
 ('b', ['44430']),
 ('gt', ['NOT', 'gs']),
 ('dp', ['dd', 'OR', 'do']),
 ('ej', ['eg', 'AND', 'ei']),
 ('ag', ['y', 'AND', 'ae']),
 ('ka', ['jx', 'AND', 'jz']),
 ('lg', ['lf', 'RSHIFT', '2']),
 ('ac', ['z', 'AND', 'aa']),
 ('el', ['dy', 'AND', 'ej']),
 ('bk', ['bj', 'OR', 'bi']),
 ('km', ['kk', 'RSHIFT', '3']),
 ('co', ['NOT', 'cn'])]

## Part 1
---

What a hot mess this one was for me...

In [2]:
def emulate(wire, wire_instructions, solved_circut):

    if wire.isdigit():
        return int(wire)

    if wire not in solved_circut:

        if len(wire_instructions[wire]) == 1:
            # E.g., lx -> a
            solved_circut[wire] = emulate(wire_instructions[wire][0], wire_instructions, solved_circut)

        elif len(wire_instructions[wire]) == 2:
            # E.g., NOT h -> i  (Is always a 'NOT' op)
            op = wire_instructions[wire][0]
            right = wire_instructions[wire][1]
            if op == 'NOT':
                solved_circut[wire] = ~ emulate(right, wire_instructions, solved_circut) & 65535

        elif len(wire_instructions[wire]) == 3:
            left = wire_instructions[wire][0]
            op = wire_instructions[wire][1]
            right = wire_instructions[wire][2]

            if op == 'AND':
                solved_circut[wire] = emulate(left, wire_instructions, solved_circut) & emulate(right, wire_instructions, solved_circut)
            elif op == 'OR':
                solved_circut[wire] = emulate(left, wire_instructions, solved_circut) | emulate(right, wire_instructions, solved_circut)
            elif op == 'LSHIFT':
                solved_circut[wire] = emulate(left, wire_instructions, solved_circut) << emulate(right, wire_instructions, solved_circut) & 65535
            elif op == 'RSHIFT':
                solved_circut[wire] = emulate(left, wire_instructions, solved_circut) >> emulate(right, wire_instructions, solved_circut)
    
    return solved_circut[wire]

This is the solution I largely based mine off of:

In [3]:
solved = {}

def solve(node):

    if node.isnumeric():
        return int(node)

    if node not in solved:

        ops = wire_instructions[node] # wires[node]

        if len(ops) == 1:
            n = solve(ops[0])

        else:
            op = ops[-2]
            if op == 'AND':
              n = solve(ops[0]) & solve(ops[2])
            elif op == 'OR':
              n = solve(ops[0]) | solve(ops[2])
            elif op == 'NOT':
              n = ~solve(ops[1]) & 65535
            elif op == 'RSHIFT':
              n = solve(ops[0]) >> solve(ops[2])
            else: #    'LSHIFT':
              n = solve(ops[0]) << solve(ops[2]) & 65535

        solved[node] = n

    return solved[node]

#Part 1
part1_solution = solve('a')
print(part1_solution)

3176


Source: https://dev.to/jules_lewis/advent-of-code-2015-day-7-35mp 

And this was my attempt before going recursive....

In [4]:
# Continue until all instructions have been processed
# processed = []
# while len(instructions) != len(processed):
    
#     # loop backwards through instructions so I can remove items from list as it's being processed.
#     for instruction in instructions:
#         if instruction not in processed:
    
#             # print(len(processed), instruction)
#             _input, _output = instruction.strip().split(' -> ')

#             if _input.isdigit():
#                 circut[_output] = int(_input)
#                 processed.append(instruction)
#             else:
#                 _input = _input.split(' ')

#             if len(_input) == 3:
#                 left = int(_input[0]) if _input[0].isdigit() else _input[0]
#                 op = _input[1]
#                 right = int(_input[2]) if _input[2].isdigit() else _input[2]

#                 if isinstance(left, int):
#                     if op == 'AND' and circut[right] != None:
#                         circut[_output] = left & circut[right]
#                         processed.append(instruction)

#                 else:
#                     if op == 'AND' and circut[left] != None and circut[right] != None:
#                         circut[_output] = circut[left] & circut[right]
#                     elif op == 'OR' and circut[left] != None and circut[right] != None:
#                         circut[_output] = circut[left] | circut[right]
#                     elif op == 'LSHIFT' and circut[left] != None:
#                         circut[_output] = circut[left] << right
#                     elif op == 'RSHIFT' and circut[left] != None:
#                         circut[_output] = circut[left] >> right

#                     processed.append(instruction)

#             elif len(_input) == 2:
#                 # print(instruction, _input[0], _input[1], circut[_input[1]])
#                 if _input[0] == 'NOT' and circut[_input[1]] != None:
#                     circut[_output] = ~ circut[_input[1]]
#                     processed.append(instruction)


### Run on Test Data

In [5]:
test_data = [
    '123 -> x',
    '456 -> y',
    'x AND y -> d',
    'x OR y -> e',
    'x LSHIFT 2 -> f',
    'y RSHIFT 2 -> g',
    'NOT x -> h',
    'NOT y -> i'
]

In [6]:
test_instructions = {}
for d in test_data:
    _input, _output = d.rstrip().split(' -> ')
    test_instructions[_output] = _input.split()

In [7]:
test_instructions

{'x': ['123'],
 'y': ['456'],
 'd': ['x', 'AND', 'y'],
 'e': ['x', 'OR', 'y'],
 'f': ['x', 'LSHIFT', '2'],
 'g': ['y', 'RSHIFT', '2'],
 'h': ['NOT', 'x'],
 'i': ['NOT', 'y']}

In [8]:
if (emulate('x', test_instructions, {}) == 123 and
    emulate('y', test_instructions, {}) == 456 and
    emulate('d', test_instructions, {}) == 72 and
    emulate('e', test_instructions, {}) == 507 and
    emulate('f', test_instructions, {}) == 492 and
    emulate('g', test_instructions, {}) == 114 and
    emulate('h', test_instructions, {}) == 65412 and
    emulate('i', test_instructions, {}) == 65079):
    print('Woot! Passed test cases.')
else:
    print('Uh oh!')

Woot! Passed test cases.


### Run on Input Data

In [9]:
emulate('a', wire_instructions, {})

3176

## Part 2
---

In [10]:
part_2_wire_instructions = wire_instructions.copy()
part_2_wire_instructions['b'] = ['3176']

emulate('a', part_2_wire_instructions, {})

14710